In [1]:
import pandas as pd
import html
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
df_generated = pd.read_csv('generated_concat.csv', index_col='Unnamed: 0')
df_generated = df_generated.reset_index(drop=True)
df_generated.head()

,title,generated_text,generated,model
0,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT
1,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT
2,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT
3,Mass Effect Trilogy на PS3 обрела дату релиза,**Mass Effect Trilogy на PS3 обрела дату релиз...,1,YandexGPT
4,Северная Корея завершила создание национальных...,**Северная Корея завершила создание национальн...,1,YandexGPT


In [3]:
df_numeric = df_generated[df_generated['title'].str.isnumeric()] 
df_numeric['title'] = df_numeric['generated_text'].str.extract('\*\*(.*)\*\*')
df_generated.loc[df_generated['title'].str.isnumeric(), 'title'] = df_numeric['title']

C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\2632328835.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['title'] = df_numeric['generated_text'].str.extract('\*\*(.*)\*\*')


In [4]:
import re

def remove_news(text):
    return text.replace('Новостная статья:', '').replace('*Примечание: В данной статье приведены гипотетические данные и информация.*', '')

def remove_between_stars(text):
    return re.sub(r'\*\*(.*?)\*\*', '', text)

def remove_title(row):
    title = row['title']
    generated_text = row['generated_text']
    index = generated_text.find(title)
    if index != -1:
        return remove_news(generated_text[:index] + generated_text[index + len(title):])
    else:
        return remove_news(remove_between_stars(generated_text))

df_generated['generated_text_without_title'] = df_generated.apply(remove_title, axis=1)

df_generated


,title,generated_text,generated,model,generated_text_without_title
0,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск..."
1,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...
2,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...
3,Mass Effect Trilogy на PS3 обрела дату релиза,**Mass Effect Trilogy на PS3 обрела дату релиз...,1,YandexGPT,"\n\nПопулярная серия игр Mass Effect, известна..."
4,Северная Корея завершила создание национальных...,**Северная Корея завершила создание национальн...,1,YandexGPT,****\n\nСеверная Корея объявила о завершении с...
...,...,...,...,...,...
16645,В суд передали дело еще четырех фигурантов «бо...,Новостная статья: В суд передали дело еще четы...,1,RuGPT3_small,\nВ суд передано дело ещё четырех обвиняемых ...
16646,Бразильский футболист попросил у Кадырова росс...,Новостная статья: Бразильский футболист попрос...,1,RuGPT3_small,и на этот раз не отказался от него\n\nВ Росс...
16647,«ВКонтакте» выкупила у Дурова дата-центр,Новостная статья: «ВКонтакте» выкупила у Дуров...,1,RuGPT3_small,в Лыткарино»\n\nВ состоянии банкротства «Вко...
16648,Захарова доверилась любви в деле восстановлени...,Новостная статья: Захарова доверилась любви в ...,1,RuGPT3_small,\n\nОригинал взят у в Захаровой доверились л...


In [5]:
titles = df_generated.title

In [6]:
def remove_non_alphanumeric(text):
    return re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s]', '', text)

def remove_html_entities(text):
    entity_pattern = re.compile(r'&[a-zA-Z]+;')
    cleaned_text = re.sub(entity_pattern, ' ', text)
    return cleaned_text

df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.lower()
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.replace('\n', ' ')
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.replace('\t', ' ')
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].apply(html.unescape)
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.replace('\xa0+', ' ')
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.replace(r'\s{2,}', ' ', regex=True)
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.replace('\*+', ' ')
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].apply(remove_non_alphanumeric)
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].apply(remove_html_entities)
df_generated['generated_text_without_title'] = df_generated['generated_text_without_title'].str.strip()

In [7]:
df_generated

,title,generated_text,generated,model,generated_text_without_title
0,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,сегодня в жилом доме расположенном в московско...
1,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,генеральная прокуратура россии провела проверк...
2,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,благотворительный фонд федерация выиграл дело ...
3,Mass Effect Trilogy на PS3 обрела дату релиза,**Mass Effect Trilogy на PS3 обрела дату релиз...,1,YandexGPT,популярная серия игр mass effect известная сво...
4,Северная Корея завершила создание национальных...,**Северная Корея завершила создание национальн...,1,YandexGPT,северная корея объявила о завершении создания ...
...,...,...,...,...,...
16645,В суд передали дело еще четырех фигурантов «бо...,Новостная статья: В суд передали дело еще четы...,1,RuGPT3_small,в суд передано дело ещё четырех обвиняемых по ...
16646,Бразильский футболист попросил у Кадырова росс...,Новостная статья: Бразильский футболист попрос...,1,RuGPT3_small,и на этот раз не отказался от него в россии в ...
16647,«ВКонтакте» выкупила у Дурова дата-центр,Новостная статья: «ВКонтакте» выкупила у Дуров...,1,RuGPT3_small,в лыткарино в состоянии банкротства вконтакте ...
16648,Захарова доверилась любви в деле восстановлени...,Новостная статья: Захарова доверилась любви в ...,1,RuGPT3_small,оригинал взят у в захаровой доверились любви в...


In [8]:
df_news = pd.read_csv('Lenta_dataset.csv', index_col='Unnamed: 0')
df_news.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,NaN
1,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,NaN
2,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,NaN
3,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,NaN
4,https://lenta.ru/news/2018/12/15/skrepy/,Российскую молодежь предложили обучать духовны...,Российская молодежь лучше усвоит духовные ценн...,Интернет и СМИ,Coцсети,NaN


In [23]:
df = df_news[df_news.title.isin(titles)]
df2 = df_news[~df_news.title.isin(titles)]

In [24]:
df2 = df2.sample(9000)

In [25]:

df['clean_text'] = df['text'].str.lower()
df['clean_text'] = df['clean_text'].str.replace('&raquash;', ' ')
df['clean_text'] = df['clean_text'].str.replace('\n+', ' ')
df['clean_text'] = df['clean_text'].str.replace('\t+', ' ')
df['clean_text'] = df['clean_text'].apply(html.unescape)
df['clean_text'] = df['clean_text'].str.replace('\xa0+', ' ')
df['clean_text'] = df['clean_text'].str.replace(r'\s{2,}', ' ', regex=True)
df['clean_text'] = df['clean_text'].str.replace('\*+', ' ')
df['clean_text'] = df['clean_text'].apply(remove_non_alphanumeric)
df['clean_text'] = df['clean_text'].apply(remove_html_entities)
df['clean_text'] = df['clean_text'].str.strip()


df2['clean_text'] = df2['text'].str.lower()
df2['clean_text'] = df2['clean_text'].str.replace('&raquash;', ' ')
df2['clean_text'] = df2['clean_text'].str.replace('\n+', ' ')
df2['clean_text'] = df2['clean_text'].str.replace('\t+', ' ')
df2['clean_text'] = df2['clean_text'].apply(html.unescape)
df2['clean_text'] = df2['clean_text'].str.replace('\xa0+', ' ')
df2['clean_text'] = df2['clean_text'].str.replace(r'\s{2,}', ' ', regex=True)
df2['clean_text'] = df2['clean_text'].str.replace('\*+', ' ')
df2['clean_text'] = df2['clean_text'].apply(remove_non_alphanumeric)
df['clean_text'] = df['clean_text'].apply(remove_html_entities)
df2['clean_text'] = df2['clean_text'].str.strip()

C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\1623072016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['text'].str.lower()
C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\1623072016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].str.replace('&raquash;', ' ')
C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\1623072016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [26]:
df['generated'] = 0
df['model'] = 'Human'

df2['generated'] = 0
df2['model'] = 'Human'

C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\447408628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['generated'] = 0
C:\Users\Alyonka\AppData\Local\Temp\ipykernel_25532\447408628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['model'] = 'Human'


In [27]:
df2

,url,title,text,topic,tags,date,clean_text,generated,model
15808,https://lenta.ru/news/2018/08/12/molodec/,Канье Уэст получил пожизненную подписку на Por...,Популярный исполнитель Канье Уэст рассказал в ...,Интернет и СМИ,Интернет,NaN,популярный исполнитель канье уэст рассказал в ...,0,Human
45112,https://lenta.ru/news/2017/12/05/freemiho/,Сторонники Саакашвили сломали дверь микроавтоб...,Сторонники Михаила Саакашвили взломали дверь м...,Бывший СССР,Закавказье,NaN,сторонники михаила саакашвили взломали дверь м...,0,Human
121400,https://lenta.ru/news/2016/09/13/nikolayev/,Мэр Николаева обвинил в сепаратизме противнико...,Мэр Николаева Александр Сенкевич обвинил депут...,Бывший СССР,Украина,NaN,мэр николаева александр сенкевич обвинил депут...,0,Human
69001,https://lenta.ru/news/2017/07/03/oil49/,Цена нефти Brent поднялась выше 49 долларов за...,Цены на нефть Brent выросли до 49 долларов за ...,Экономика,Госэкономика,NaN,цены на нефть brent выросли до 49 долларов за ...,0,Human
60567,https://lenta.ru/news/2017/08/21/mayo/,Журналист добавил в кофе майонез и вызвал омер...,Американский спортивный журналист Джим Сэйлсбе...,Интернет и СМИ,Интернет,NaN,американский спортивный журналист джим сэйлсбе...,0,Human
...,...,...,...,...,...,...,...,...,...
111119,https://lenta.ru/news/2016/11/08/ryanair_job/,Крупнейший лоукостер Европы предложил «худшую ...,Крупнейшая бюджетная авиакомпания Европы Ryana...,Путешествия,Мир,NaN,крупнейшая бюджетная авиакомпания европы ryana...,0,Human
137410,https://lenta.ru/news/2016/06/14/eddie/,Исполнитель роли Стивена Хокинга снялся в рекл...,Актер Эдди Рэдмейн снялся в рекламной кампании...,Ценности,Явления,NaN,актер эдди рэдмейн снялся в рекламной кампании...,0,Human
110783,https://lenta.ru/news/2016/11/09/domagoyvidast...,Капитана киевского «Динамо» поймали пьяным за ...,Капитан украинского «Динамо» Домагой Вида был ...,Спорт,Футбол,NaN,капитан украинского динамо домагой вида был за...,0,Human
129613,https://lenta.ru/news/2016/07/28/klukovka/,Путин призвал делать из российской клюквы не т...,"Президент России Владимир Путин считает, что и...",Россия,Политика,NaN,президент россии владимир путин считает что из...,0,Human


In [28]:
merged_df = pd.merge(df_generated, df[['title', 'clean_text']], on='title', how='left')

In [29]:
merged_df

,title,generated_text,generated,model,generated_text_without_title,clean_text
0,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,сегодня в жилом доме расположенном в московско...,NaN
1,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...
2,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,благотворительный фонд федерация выиграл дело ...,NaN
3,Mass Effect Trilogy на PS3 обрела дату релиза,**Mass Effect Trilogy на PS3 обрела дату релиз...,1,YandexGPT,популярная серия игр mass effect известная сво...,NaN
4,Северная Корея завершила создание национальных...,**Северная Корея завершила создание национальн...,1,YandexGPT,северная корея объявила о завершении создания ...,северная корея в 2017 году завершила создание ...
...,...,...,...,...,...,...
16693,В суд передали дело еще четырех фигурантов «бо...,Новостная статья: В суд передали дело еще четы...,1,RuGPT3_small,в суд передано дело ещё четырех обвиняемых по ...,NaN
16694,Бразильский футболист попросил у Кадырова росс...,Новостная статья: Бразильский футболист попрос...,1,RuGPT3_small,и на этот раз не отказался от него в россии в ...,NaN
16695,«ВКонтакте» выкупила у Дурова дата-центр,Новостная статья: «ВКонтакте» выкупила у Дуров...,1,RuGPT3_small,в лыткарино в состоянии банкротства вконтакте ...,NaN
16696,Захарова доверилась любви в деле восстановлени...,Новостная статья: Захарова доверилась любви в ...,1,RuGPT3_small,оригинал взят у в захаровой доверились любви в...,официальный представитель мид россии мария зах...


In [30]:
generated = df_generated[['title', 'generated', 'model', 'generated_text',  'generated_text_without_title']].rename(columns={'generated_text':'text', 'generated_text_without_title':'clean_text'})
generated

,title,generated,model,text,clean_text
0,Пожар в жилом доме в Солнцево потушен,1,YandexGPT,**Пожар в жилом доме в Солнцево потушен**\n\nС...,сегодня в жилом доме расположенном в московско...
1,В Генпрокуратуре заявили о наличии обманутых д...,1,YandexGPT,**В Генпрокуратуре заявили о наличии обманутых...,генеральная прокуратура россии провела проверк...
2,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",1,YandexGPT,**Фонд «Федерация» отсудил у «Коммерсанта» три...,благотворительный фонд федерация выиграл дело ...
3,Mass Effect Trilogy на PS3 обрела дату релиза,1,YandexGPT,**Mass Effect Trilogy на PS3 обрела дату релиз...,популярная серия игр mass effect известная сво...
4,Северная Корея завершила создание национальных...,1,YandexGPT,**Северная Корея завершила создание национальн...,северная корея объявила о завершении создания ...
...,...,...,...,...,...
16645,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...
16646,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...
16647,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...
16648,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...


In [31]:
tmp = df2[['title', 'generated', 'model', 'text', 'clean_text']]
news = pd.concat([tmp,df[['title', 'generated', 'model', 'text', 'clean_text']]])
news

,title,generated,model,text,clean_text
15808,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...
45112,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...
121400,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...
69001,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...
60567,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...
...,...,...,...,...,...
161167,В Ростовской области убили охранника при неуда...,0,Human,В городе Шахты Ростовской области трое неизвес...,в городе шахты ростовской области трое неизвес...
161176,Райан Рейнольдс назвал россиян самыми крутыми ...,0,Human,"Голливудский актер Райан Рейнольдс, сыгравший ...",голливудский актер райан рейнольдс сыгравший д...
161188,Шарлотта Рэмплинг отказалась от слов о недосто...,0,Human,"Британская актриса Шарлотта Рэмплинг, номиниро...",британская актриса шарлотта рэмплинг номиниров...
161220,Jimmy Choo представил свадебную коллекцию,0,Human,Британский производитель обуви Jimmy Choo пред...,британский производитель обуви jimmy choo пред...


In [32]:
dataset = pd.concat([news, generated], ignore_index=True)
dataset

,title,generated,model,text,clean_text
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...
...,...,...,...,...,...
32870,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...
32871,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...
32872,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...
32873,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...


In [33]:
nltk.data.path.append("D:/Programs/nltk_data")

def sentence_stat(text): 
    sentences = sent_tokenize(text)
    avg_sentence_len = sum(len(s) for s in sentences) / len(sentences)
    return avg_sentence_len

def words_stat(text):
    words = word_tokenize(text)
    avg_words_len = sum(len(w) for w in words) / len(words)
    return avg_words_len


dataset['avg_sentence_len'] = dataset['text'].apply(sentence_stat)
dataset['avg_words_len'] = dataset['text'].apply(words_stat)

dataset

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846
...,...,...,...,...,...,...,...
32870,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000
32871,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142
32872,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534
32873,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568


In [34]:
dataset['char_count'] = dataset['clean_text'].apply(len)
dataset['word_count'] = dataset['clean_text'].apply(lambda x: len(x.split()))
dataset['sentence_count'] = dataset['text'].apply(lambda x: x.count('.') + x.count('!') + x.count('?'))

dataset.drop(dataset[dataset.char_count<100].index, axis=0)

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968,1105,151,14
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000,1208,162,12
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000,1221,163,14
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383,1162,179,10
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846,1174,163,9
...,...,...,...,...,...,...,...,...,...,...
32870,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,2562,312,37
32871,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1113,159,61
32872,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,1197,160,36
32873,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,966,147,54


In [35]:
dataset.sample(10)

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
31538,ЦБ обнаружил признаки вывода активов из Идея-б...,1,RuGPT3_small,Новостная статья: ЦБ обнаружил признаки вывода...,появилась на фоне продолжающихся событий связа...,124.176471,4.885333,1974,284,17
12655,Тарпищев предрек возвращение Шараповой в элиту...,0,Human,Президент Федерации тенниса России (ФТР) Шамил...,президент федерации тенниса россии фтр шамиль ...,90.117647,4.846715,1490,216,17
18040,Уязвимость в Linux поставила под удар десятки ...,1,YandexGPT,**Уязвимость в Linux поставила под удар десятк...,исследователи по кибербезопасности обнаружили ...,156.125000,6.356322,1161,141,8
1342,Млечный Путь начал умирать,0,Human,Астрофизики из Франции и США представили перво...,астрофизики из франции и сша представили перво...,136.500000,5.409091,1068,146,8
20726,Названы самые популярные туристические направл...,1,YandexGPT lite,Названы самые популярные направления туризма ...,названы самые популярные направления туризма у...,178.000000,4.302857,828,116,5
32796,Американские и украинские военные выполнят наб...,1,RuGPT3_small,Новостная статья: Американские и украинские во...,с российской территории украинские и российски...,89.000000,4.789137,3005,396,96
6011,Совфед одобрил отмену открепительных удостовер...,0,Human,Совет Федерации в ходе заседания одобрил блок ...,совет федерации в ходе заседания одобрил блок ...,137.125000,5.674556,1081,146,8
12958,Военнослужащие Перу эвакуировали жертв наводне...,0,Human,"Военнослужащие армии Перу спасли людей, которы...",военнослужащие армии перу спасли людей которые...,88.500000,5.722222,349,49,4
7292,В Южной Корее задержан глава пенсионного фонда,0,Human,"Специальная прокуратура Южной Кореи, расследую...",специальная прокуратура южной кореи расследующ...,107.636364,5.718232,1170,160,11
16484,На месте хлебозавода построят дорогие офисы,1,YandexGPT,**На месте хлебозавода построят дорогие офисы*...,в центре города на месте бывшего хлебозавода п...,108.727273,5.977273,1138,148,11


In [41]:
dataset.groupby(by=['model'])[['char_count', 'word_count', 'sentence_count','avg_sentence_len','avg_words_len']].agg(['min', 'max', 'mean', 'median'])

char_count                            word_count       \
                      min   max         mean  median        min  max   
model                                                                  
GigaChat              587  2704  1265.465080  1248.0         88  404   
Human                 202  7178  1194.029276  1132.0         24  919   
RuGPT3_small          216  5662  1520.977366  1380.5         30  738   
YandexGPT             300  3028  1009.459069   985.0         37  384   
YandexGPT lite        149  3283  1110.825157  1080.0         20  450   

                                  sentence_count                        \
                      mean median            min max       mean median   
model                                                                    
GigaChat        182.465302  179.0              2  39  11.589635   11.0   
Human           166.587550  159.0              1  93  11.613005   11.0   
RuGPT3_small    219.020919  201.0              2  92  24.489712   22.0   
YandexGPT       135.693419  132.0              4  51  10.327219   10.0   
YandexGPT lite  151.037945  146.0              1  49  10.694130   10.0   

               avg_sentence_len                                       \
                            min          max        mean      median   
model                                                                  
GigaChat              52.423077   838.666667  115.753767  114.200000   
Human                 41.200000   254.454545  110.464711  108.000000   
RuGPT3_small          17.701493  1316.000000  142.705669  120.289583   
YandexGPT             39.269231   215.333333  106.155138  104.875000   
YandexGPT lite        34.480000   302.333333  118.679718  115.358974   

               avg_words_len                                
                         min       max      mean    median  
model                                                       
GigaChat            3.381232  6.729469  5.189755  5.183333  
Human               3.311966  7.314103  5.284955  5.283843  
RuGPT3_small        1.936281  9.966667  4.086297  4.104842  
YandexGPT           3.535433  7.378049  5.576580  5.577778  
YandexGPT lite      3.077670  7.736264  5.446893  5.433430

In [37]:
dataset = dataset.drop(dataset[dataset.sentence_count>93].index)

In [40]:
dataset = dataset.drop(dataset[dataset.sentence_count==0].index)
dataset = dataset.drop(dataset[dataset.word_count<20].index)

In [42]:
dataset

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968,1105,151,14
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000,1208,162,12
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000,1221,163,14
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383,1162,179,10
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846,1174,163,9
...,...,...,...,...,...,...,...,...,...,...
32870,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,2562,312,37
32871,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1113,159,61
32872,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,1197,160,36
32873,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,966,147,54


In [43]:
dataset.to_csv('cooked_dataset.csv')

In [3]:
df = pd.read_csv('cooked_dataset.csv', index_col='Unnamed: 0').reset_index(drop=True)
df.tail()

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,2562,312,37
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1113,159,61
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,1197,160,36
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,966,147,54
32767,Женщина залезла в канализацию ради спасения утят,1,RuGPT3_small,Новостная статья: Женщина залезла в канализаци...,в новосибирске в одном из отелей где проходили...,128.312500,2.472000,1283,196,32


In [4]:
from sklearn.model_selection import train_test_split

In [7]:
train, test= train_test_split(
    df, test_size=0.2, random_state=42
)

In [9]:
train.to_csv('cooked_dataset_train.csv')
test.to_csv('cooked_dataset_test.csv')